In [220]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "01" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 25
repetitionName = "udrlBC2"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.5
print(len(savedResults))
print(len(savedResults)/9)

9
1.0


In [221]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()


# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




(1, 135)
(10, 135)


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_8056\489148831.py:172: RuntimeWarning: invalid value encountered in divide
  comboAverage = comboAverage / nrOfSubjects


,0,1,2,6,7,8,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,63,64,65,66,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,96,97,98,99,100,101,102,103,104,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,0.7,0.7,0.675,0.575,0.625,0.625,0.625,0.625,0.725,0.625,0.65,0.7,0.575,None,0.5,0.7,0.725,0.675,0.625,0.7,0.675,0.675,0.7,0.65,0.7,0.7,0.65,0.7,0.7,0.675,0.7,0.7,0.725,0.7,0.65,0.7,0.75,0.65,0.7,0.6,0.65,0.6,0.625,0.675,0.75,None,0.55,None,0.55,0.575,0.625,0.625,0.6,0.7,0.625,0.575,0.625,0.625,0.75,0.7,0.775,0.6,0.7,0.65,0.7,0.7,0.675,0.625,0.65,0.725,0.675,0.675,0.675,0.625,None,None,0.625,0.675,0.725,0.5,None,None,0.575,None,0.5,0.625,0.625,0.725,0.6,0.7,0.7,0.575,None,None,0.625,0.675,0.7,0.725,0.625,0.675,0.6,0.575,0.7
2,0.75,0.791667,0.75,0.770833,0.75,0.729167,0.791667,0.791667,0.875,0.604167,0.770833,0.791667,0.708333,None,0.666667,0.75,0.8125,0.770833,0.75,0.791667,0.75,0.75,0.770833,0.770833,0.791667,0.8125,0.791667,0.75,0.708333,0.791667,0.854167,0.770833,0.833333,0.75,0.708333,0.791667,0.770833,0.75,0.791667,0.770833,0.770833,0.729167,0.791667,0.791667,0.875,None,0.708333,None,0.666667,0.770833,0.770833,0.729167,0.791667,0.75,0.75,0.729167,0.729167,0.708333,0.770833,0.708333,0.791667,0.770833,0.6875,0.729167,0.729167,0.791667,0.75,0.770833,0.791667,0.854167,0.6875,0.75,0.791667,0.729167,None,0.666667,0.8125,0.666667,0.854167,0.583333,None,None,0.604167,None,0.625,0.791667,0.6875,0.833333,0.604167,0.75,0.791667,0.729167,None,0.645833,0.791667,0.6875,0.833333,0.75,0.604167,0.833333,0.791667,0.666667,0.770833
3,0.638889,0.666667,0.666667,0.722222,0.722222,0.777778,0.583333,0.611111,0.694444,0.611111,None,0.666667,0.5,None,None,0.638889,0.666667,0.666667,0.75,0.694444,0.694444,0.638889,0.666667,0.694444,0.638889,0.638889,0.666667,0.638889,0.722222,0.666667,0.611111,0.777778,0.666667,0.638889,0.722222,0.666667,0.694444,0.694444,0.666667,0.722222,0.75,0.777778,0.611111,0.611111,0.666667,None,0.5,None,None,0.722222,0.722222,0.805556,0.805556,0.722222,0.722222,0.75,0.638889,0.666667,0.722222,0.805556,0.694444,0.75,0.75,0.722222,0.666667,0.722222,0.666667,0.583333,0.638889,0.694444,0.694444,None,0.638889,0.583333,0.5,None,0.666667,0.694444,0.638889,0.638889,None,None,0.638889,0.527778,None,0.583333,0.694444,0.694444,0.611111,0.5,None,0.527778,None,None,0.583333,0.694444,0.694444,0.722222,0.666667,0.638889,0.583333,0.583333,0.5
4,0.770833,0.770833,0.791667,0.75,0.708333,0.75,0.6875,0.666667,0.708333,0.645833,0.708333,0.708333,0.583333,None,0.583333,0.770833,0.770833,0.791667,0.75,0.75,0.770833,0.770833,0.770833,0.791667,0.770833,0.6875,0.75,0.770833,0.770833,0.791667,0.791667,0.770833,0.791667,0.770833,0.770833,0.791667,0.708333,0.666667,0.645833,0.729167,0.729167,0.75,0.708333,0.666667,0.708333,None,0.583333,None,0.5625,0.75,0.729167,0.75,0.770833,0.729167,0.791667,0.75,0.75,0.75,0.770833,0.770833,0.770833,0.791667,0.770833,0.729167,0.729167,0.5625,0.666667,0.75,0.6875,0.708333,0.6875,0.729167,0.708333,0.6875,None,0.583333,0.6875,0.604167,0.791667,0.708333,0.604167,0.75,0.5625,0.520833,0.5625,0.6875,0.729167,0.6875,0.645833,0.75,0.708333,0.604167,None,0.520833,0.666667,0.791667,0.6875,0.5625,0.5625,0.645833,0.666667,0.604167,0.666667
5,0.770833,0.666667,0.6875,0.8125,0.6875,0.75,0.666667,0.625,0.708333,0.729167,0.6875,0.75,0.6875,0.541667,0.729167,0.770833,0.666667,0.6875,0.75,0.6875,0.

In [222]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.88 : subject 5, feature:dataFFTCV--welchDataBC, kernellinear, C:0.5
Accuracy 0.88 : subject 2, feature:dataFFTCV-BC--fftData, kernelrbf, C:2.5
Accuracy 0.88 : subject 2, feature:-dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataFFTCV-BC--welchData, kernelrbf, C:2.5
Accuracy 0.85 : subject 2, feature:dataFFTCV-BC--fftDataBC, kernellinear, C:0.5
Accuracy 0.85 : subject 2, feature:dataFFTCV-BC--dataFFTCV, kernelrbf, C:2.5
Accuracy 0.83 : subject 5, feature:welchDataBC--fftData, kernellinear, C:0.5
Accuracy 0.83 : subject 2, feature:dataWCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.83 : subject 2, feature:dataHRCV-BC--dataFFTCV-BC, kernelrbf, C:2.5
Accuracy 0.83 : subject 2, feature:dataFFTCV-BC--fftDataBC, kernelrbf, C:2.5


In [223]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)


print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.65564516 0.75197368 0.66510612 0.70727041 0.70118464
 0.6674397  0.69828431 0.67718447 0.64492754]


,0,1,2,6,7,8,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,63,64,65,66,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,96,97,98,99,100,101,102,103,104,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134
0,0.723555,0.713005,0.715067,0.711055,0.708319,0.71445,0.639941,0.670991,0.6953,0.622868,0.686506,0.701641,0.589232,0.618434,0.624558,0.724018,0.718098,0.714857,0.72016,0.723036,0.717691,0.730499,0.717635,0.717691,0.720356,0.699818,0.688931,0.723555,0.721072,0.726431,0.723288,0.746815,0.715572,0.723555,0.720567,0.724579,0.702455,0.682758,0.687332,0.711518,0.730808,0.706832,0.663608,0.682686,0.697727,0.55,0.611255,0.660732,0.62279,0.706215,0.715474,0.710382,0.732856,0.701922,0.684708,0.715993,0.70369,0.711574,0.707898,0.726333,0.726487,0.72479,0.731524,0.717999,0.682029,0.688412,0.667677,0.659848,0.680193,0.690881,0.676436,0.707792,0.707229,0.652597,0.592519,0.63947,0.656818,0.659249,0.686813,0.621627,0.590278,0.701389,0.617593,0.597525,0.597096,0.642719,0.678567,0.688356,0.608558,0.642508,0.693561,0.588709,0.572854,0.60197,0.635774,0.697838,0.687893,0.63166,0.634564,0.680477,0.62571,0.642282,0.666241


Max average accuracy 0.7468153759820427
43
9.0
welchDataBC--fftDataBC
sigmoid
C: 2.5


In [224]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-fftDataBC
1
-fftData
2
-welchDataBC
3
-welchData
4
-dataFFTCV
5
-dataWCV
6
-dataFFTCV-BC
7
-dataWCV-BC
8
-dataHRCV-BC
9
fftData--fftDataBC
10
welchDataBC--fftDataBC
11
welchData--fftDataBC
12
dataFFTCV--fftDataBC
13
dataWCV--fftDataBC
14
dataFFTCV-BC--fftDataBC
15
dataWCV-BC--fftDataBC
16
dataHRCV-BC--fftDataBC
17
welchDataBC--fftData
18
welchData--fftData
19
dataFFTCV--fftData
20
dataWCV--fftData
21
dataFFTCV-BC--fftData
22
dataWCV-BC--fftData
23
dataHRCV-BC--fftData
24
welchData--welchDataBC
25
dataFFTCV--welchDataBC
26
dataWCV--welchDataBC
27
dataFFTCV-BC--welchDataBC
28
dataWCV-BC--welchDataBC
29
dataHRCV-BC--welchDataBC
30
dataFFTCV--welchData
31
dataWCV--welchData
32
dataFFTCV-BC--welchData
33
dataWCV-BC--welchData
34
dataHRCV-BC--welchData
35
dataWCV--dataFFTCV
36
dataFFTCV-BC--dataFFTCV
37
dataWCV-BC--dataFFTCV
38
dataHRCV-BC--dataFFTCV
39
dataFFTCV-BC--dataWCV
40
dataWCV-BC--dataWCV
41
dataHRCV-BC--dataWCV
42
dataWCV-BC--dataFFTCV-BC
43
dataHRCV-BC--dataFFTCV-BC
44
dataHRCV-B

In [225]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,0,1,2,6,7,8,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,63,64,65,66,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,96,97,98,99,100,101,102,103,104,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134
0,6.511995,6.417045,6.435606,6.399495,6.374874,6.430051,5.75947,5.367929,6.257702,5.605808,5.492045,6.314773,4.124621,1.855303,3.747348,6.516162,6.462879,6.433712,6.481439,6.507323,6.459217,6.574495,6.458712,6.459217,6.483207,6.298359,6.200379,6.511995,6.489646,6.537879,6.509596,6.721338,6.440152,6.511995,6.485101,6.521212,6.322096,6.144823,6.185985,6.403662,6.577273,6.36149,5.972475,5.46149,6.279545,0.55,4.278788,1.982197,3.736742,6.355934,6.439268,6.393434,6.595707,6.317298,6.162374,6.443939,6.333207,6.404167,6.371086,6.536995,6.538384,6.523106,6.583712,6.461995,6.138258,6.195707,6.009091,5.938636,5.44154,6.217929,5.41149,4.954545,5.657828,4.568182,2.370076,3.197348,5.911364,5.27399,6.181313,4.351389,1.770833,2.104167,3.705556,2.987626,3.582576,5.78447,5.428535,6.195202,5.47702,5.782576,5.548485,4.12096,1.718561,3.009848,5.72197,5.582702,6.191035,5.053283,5.076515,5.443813,5.005682,5.138258,5.329924


43
9.0
Max total significant accuracy then averaged 0.7468153759820427 
welchDataBC--fftDataBC
sigmoid
C: 2.5 


In [226]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
